# Convolutional Long Short Term Memory neural network

## Introduction

This notebook summarizes the **dataset**, the **ConvLSTM** model architecture, the current **training loop** used for predicting wildfire spread, and how we convert model probabilities on the grid into a pixel graph and adjacency matrix that will later feed a QUBO formulation.

In [9]:

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import sys

In [25]:
%cd ../..
from ml.data.split_data import FireSpreadDataset

/Users


### Dataset

The dataset used gathers satelite data from Google Earth. It is a time series of daily observations which have active fires and extra variables such as fuel, topography and weather conditions. It is available at [Zenodo](https://zenodo.org/records/8006177). It contains GeoTIFF files which will be used to train the proposed model. In the following picture it is displayed a specific tif with the fire that is captured and shown in band **23**.

<img src="../../images/tif.jpeg" alt="Heatmap" width="400">

## Model architecture

A multivariate **spatio-temporal** model is implemented so as to make use of both temporal and spatial dependencies that are natural of the data. It takes into account the time frames because the fire spreding depends on the time we are looking at, and the spatial dimension because of the location of the fire spreading. The following architecture is used. 


```
Time axis →
t-(T-1)        t-(T-2)              ...                  t-1
   │               │                                      │
   ├──── Frame ────┼─────────────── … ────────────────────┤   (B, C, H, W)
   ▼               ▼                                      ▼
┌───────────────────────────────────────────────────────────────┐
│                     ConvLSTM Layer 1                          │
│  (process each time step; pass hidden state through time)     │
└───────────────────────────────────────────────────────────────┘
                               │ (per time step output h¹_t)
                               ▼
┌───────────────────────────────────────────────────────────────┐
│                     ConvLSTM Layer 2                          │
└───────────────────────────────────────────────────────────────┘
                               │ (per time step output h²_t)
                               ▼
┌───────────────────────────────────────────────────────────────┐
│                     ConvLSTM Layer 3                          │
└───────────────────────────────────────────────────────────────┘
                               │
                         (take last time step output h³_t-1)
                               ▼
                    1×1 Conv (output head)
                               ▼
                      Next-frame map (B, 1, C, H, W)
                          (e.g., fire probability)
```


### Training

After training the model, we get the heatmap of probabilites which are dispersed in the pixels of the frame. After, the adyacency matrix is used based on the prdictions of the model, which will be the input of QUBO.

<img src="../../images/heat_map.png" alt="Heatmap" width="600">